In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import xarray as xr
import torch

In [3]:
baker_url = 's3://petrichor/geosmart/baker.zarr/'
scg_url = 's3://petrichor/geosmart/scg.zarr/'

In [4]:
baker_ds = xr.open_dataset(baker_url, chunks='auto', engine='zarr', storage_options={"anon": True})
scg_ds = xr.open_dataset(scg_url, chunks='auto', engine='zarr', storage_options={"anon": True})

In [5]:
baker_ds

<xarray.Dataset>
Dimensions:      (time: 55, y: 12089, x: 11383)
Coordinates:
  * time         (time) datetime64[ns] 1947-09-14 1950-09-02 ... 2020-12-12
  * x            (x) float64 5.809e+05 5.809e+05 ... 5.923e+05 5.923e+05
  * y            (y) float64 5.409e+06 5.409e+06 ... 5.397e+06 5.397e+06
Data variables:
    band1        (time, y, x) float32 dask.array<chunksize=(55, 712, 670), meta=np.ndarray>
    spatial_ref  int64 ...
Attributes:
    AREA_OR_POINT:  Area
    add_offset:     0.0
    crs:            PROJCS["WGS 84 / UTM zone 10N",GEOGCS["WGS 84",DATUM["WGS...
    scale_factor:   1.0

In [6]:
# Convert to array
baker_array = baker_ds.to_array(dim='variable')


In [27]:
baker_array.coords['time'].values

# Convert timestamps to unix time, normalize by starting at 0 and also do it in years
years = baker_array.coords['time'].astype('int64') / 1e9 / 60 / 60 / 24 / 365

# Start at 0
years = years - years[0]

In [54]:
# Convert time to numpy array
years_tens = torch.tensor(years.values)

# Convert coordinates to tensors
x_tens = torch.tensor(baker_array.coords['x'].values)
y_tens = torch.tensor(baker_array.coords['y'].values)

# Convert elevation to tensor
# elev_tens = torch.tensor(baker_array.

In [59]:
baker_array.sel(variable='band1')

<xarray.DataArray (time: 55, y: 12089, x: 11383)>
dask.array<getitem, shape=(55, 12089, 11383), dtype=float64, chunksize=(55, 552, 552), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 1947-09-14 1950-09-02 ... 2020-12-12
  * x         (x) float64 5.809e+05 5.809e+05 5.809e+05 ... 5.923e+05 5.923e+05
  * y         (y) float64 5.409e+06 5.409e+06 5.409e+06 ... 5.397e+06 5.397e+06
    variable  <U5 'band1'
Attributes:
    AREA_OR_POINT:  Area
    add_offset:     0.0
    crs:            PROJCS["WGS 84 / UTM zone 10N",GEOGCS["WGS 84",DATUM["WGS...
    scale_factor:   1.0

In [13]:
baker_array.coords['x'].values

array([580947.43871192, 580948.43871192, 580949.43871192, ...,
       592327.43871192, 592328.43871192, 592329.43871192])